## Player data

In [1]:
## for calling dataset
import requests

## parsing dataset
from chicken_dinner.pubgapi import PUBG
from chicken_dinner.constants import COLORS
from chicken_dinner.constants import map_dimensions

import json
import numpy as np
import pandas as pd

from tqdm import tqdm
from datetime import datetime, timedelta
import glob
import config

In [2]:
api_key = config.api_key

In [71]:
today = datetime.now() - timedelta(1)
date = today.strftime('%Y-%m-%d')
date = '2024-02-09'

In [72]:
# flist = glob.glob(f'data/{today}*_smp_data.json')
flist = glob.glob(f'data/{date}/*_smp_data.json')
flist

['data/2024-02-09/2024-02-09_smp_data.json']

In [73]:
## save data into json file
sample_file_path = flist[0]
sample_file_path

'data/2024-02-09/2024-02-09_smp_data.json'

In [74]:
with open(sample_file_path, 'r') as f:
    sample_data = json.load(f)

In [75]:
match_list = sample_data['data']['relationships']['matches']['data']
len(match_list), match_list[:5]

(171,
 [{'type': 'match', 'id': '8cc6c687-657b-48a0-8282-9e6b7e16d2ce'},
  {'type': 'match', 'id': 'ef645db7-a848-416c-89de-85f272b993df'},
  {'type': 'match', 'id': 'd892a132-a581-40a6-8354-290f808a3595'},
  {'type': 'match', 'id': '092188b5-ddf7-4c87-9b99-e929c36e9769'},
  {'type': 'match', 'id': 'c37c341f-d968-4374-804c-7c2cdd3a62a6'}])

#### get player data

In [13]:
features = ['player_name', 'player_id', 'match_id', 'map', 'rank',
            'player_position',   ## 'battle_time', 'battle_loc', 
            'death_loc', 'survival_time']

In [14]:
player_df = pd.DataFrame(columns = features)
player_df

,player_name,player_id,match_id,map,rank,player_position,death_loc,survival_time


In [10]:
pubg = PUBG(api_key, shard = 'kakao')

In [ ]:
error_list = []

for i, match in tqdm(enumerate(match_list)):
    print(player_df.shape, end='\r')
    try:
        # pubg = PUBG(api_key, shard = 'kakao')
        match_id = match['id']
        current_match = pubg.match(match_id)
        telemetry = current_match.get_telemetry()
        map = telemetry.map_name()

        player_list = telemetry.player_ids()
        
        for ply in player_list:
            try:                
                player_name = pubg.players_from_ids(ply).names[0]
                player_id = ply
                player_position = telemetry.player_positions()[player_name]

                rank = [x for x in telemetry.rankings().keys() if player_name in telemetry.rankings()[x]][0]
                
                # battle_time = None
                # battle_loc = [None, None]
                death_loc = [None, None]
                survival_time = None

                player_df.loc[len(player_df)] = [player_name, player_id, match_id, map,
                                                 rank, player_position, death_loc,
                                                 survival_time]
            except:
                # print(ply)
                error_list.append([match_id, ply])
    except:
        print(match)
        error_list.append(match)
        

0it [00:00, ?it/s]

1it [01:33, 93.20s/it]

In [ ]:
pubg = PUBG(api_key, shard = 'kakao')

In [ ]:
for error in error_list:
    try:
        match_id = error[0]
        current_match = pubg.match(match_id)
        telemetry = current_match.get_telemetry()
        map = telemetry.map_name()
        
        player_id = error[1]
        player_name = str(pubg.players_from_ids(player_id).names[0])
        player_position = telemetry.player_positions()[player_name]
    
        rank = [x for x in telemetry.rankings().keys() if player_name in telemetry.rankings()[x]][0]
        
        # battle_time = None
        # battle_loc = [None, None]
        death_loc = [None, None]
        survival_time = None
    
        player_df.loc[len(player_df)] = [player_name, player_id, match_id, map,
                                         rank, player_position, death_loc,
                                         survival_time]
    except:
        # print(error)
        continue

In [ ]:
print(player_df.shape)
player_df.tail()

In [ ]:
for i, match in tqdm(enumerate(match_list)):
    # try:
    pubg = PUBG(api_key, shard = 'kakao')
    match_id = match['id']
    current_match = pubg.match(match_id)
    telemetry = current_match.get_telemetry()
    log_list = telemetry.filter_by(event_type = 'log_player_kill_v2')
    
    startTime = pd.to_timedelta(telemetry.started()[telemetry.started().find('T')+1:-1])

    for log in log_list:
        name = log['victim']['name']
        loc = [log['victim']['location']['x'], log['victim']['location']['y']]
        endTime = log.timestamp
        endTime = (pd.to_timedelta(endTime[endTime.find('T')+1:-1]) - startTime).total_seconds()
        try:
            ply_idx = player_df['player_name'].tolist().index(name)
    
            player_df.at[ply_idx, 'death_loc'] = loc
            player_df.at[ply_idx, 'survival_time'] = endTime
        except:
            continue
    # except:
    #     print(match['id'])

In [ ]:
print(player_df.shape)
player_df.tail()

#### save the player log data

In [ ]:
json_data = player_df.to_json(orient='records')

In [ ]:
json_file_name = f'data/{date}/{date}_ply_data.json'
json_file_name

In [ ]:
with open(json_file_name, 'w') as f:
    json.dump(json_data, f)

#### player battle log data (교전 데이터)

In [76]:
features_b = ['player_name', 'player_id', 'match_id', 'map', 
            'battle_time', 'battle_loc', 'phase']

In [77]:
battle_df = pd.DataFrame(columns = features_b)
battle_df

,player_name,player_id,match_id,map,battle_time,battle_loc,phase


In [78]:
def get_phase(x):
    return sum([phase <= x for phase in phase_time]) +1

In [79]:
error_list_2 = []
pubg = PUBG(api_key, shard = 'kakao')
for i, match in tqdm(enumerate(match_list)):
    try:
        match_id = match['id']
        current_match = pubg.match(match_id)
        telemetry = current_match.get_telemetry()
        map = telemetry.map_name()
        startTime = pd.to_timedelta(telemetry.started()[telemetry.started().find('T')+1:-1])

        ## get phase info
        circles = telemetry.circle_positions()
        whites = np.array(circles['white'])
        phases = np.where(whites[1:, 4] - whites[:-1, 4] != 0)[0] + 1
        
        phase_time = []
        for phase in phases:
            phase_time.append(whites[phase][0])

        ## get battle logs
        attackLog = telemetry.filter_by('log_player_attack') # 교전 (공격한 경우) 데이터

        attackData = [(match_id, map, log['attacker']['name'], log['attacker']['location']['x'], log['attacker']['location']['y'],
               (pd.to_timedelta(log.timestamp[log.timestamp.find('T')+1:-1]) - startTime).total_seconds(),
              log['attacker']['team_id'])
                for log in attackLog if pd.to_timedelta(log.timestamp[log.timestamp.find('T')+1:-1]) > startTime]

        if i == 0:
            attack_df = pd.DataFrame(attackData, columns=['match_id', 'map', 'name', 
                                                           'loc_x', 'loc_y', 'time', 'team_id'])
            attack_df['phase'] = attack_df['time'].map(get_phase)
        else:
            attackData = pd.DataFrame(attackData, columns=['match_id', 'map', 'name', 
                                                           'loc_x', 'loc_y', 'time', 'team_id'])
            attackData['phase'] = attackData['time'].map(get_phase)
            attack_df = pd.concat([attack_df, attackData], ignore_index = True)

        # print(attack_df.shape, end = '\r')
    except:
        print(match_id)
        error_list_2.append(match_id)

31it [03:08,  6.74s/it]

f7fab979-b71c-486b-8e89-818588598ab1


73it [08:17,  4.89s/it]

8f14d554-c335-4801-ba7e-37eee30ffb30


116it [13:02,  9.75s/it]

f590fe8d-9aa6-43e0-bf5b-dc404764e7e7


171it [21:13,  7.45s/it]


In [80]:
print(attack_df.shape)
attack_df.head()

(477561, 8)


,match_id,map,name,loc_x,loc_y,time,team_id,phase
0,8cc6c687-657b-48a0-8282-9e6b7e16d2ce,Camp Jackal,angkimozzieeeeee,126706.976562,138285.625,54.43548,1,1
1,8cc6c687-657b-48a0-8282-9e6b7e16d2ce,Camp Jackal,angkimozzieeeeee,126706.976562,138285.625,54.53548,1,1
2,8cc6c687-657b-48a0-8282-9e6b7e16d2ce,Camp Jackal,angkimozzieeeeee,126706.976562,138285.625,54.56848,1,1
3,8cc6c687-657b-48a0-8282-9e6b7e16d2ce,Camp Jackal,angkimozzieeeeee,126706.976562,138285.625,54.63548,1,1
4,8cc6c687-657b-48a0-8282-9e6b7e16d2ce,Camp Jackal,angkimozzieeeeee,126706.976562,138285.625,54.73648,1,1


In [81]:
pubg = PUBG(api_key, shard = 'kakao')
for i, match_id in tqdm(enumerate(error_list_2)):
    try:
        current_match = pubg.match(match_id)
        telemetry = current_match.get_telemetry()
        map = telemetry.map_name()
        startTime = pd.to_timedelta(telemetry.started()[telemetry.started().find('T')+1:-1])

        ## get phase info
        circles = telemetry.circle_positions()
        whites = np.array(circles['white'])
        phases = np.where(whites[1:, 4] - whites[:-1, 4] != 0)[0] + 1
        
        phase_time = []
        for phase in phases:
            phase_time.append(whites[phase][0])

        ## get battle logs
        attackLog = telemetry.filter_by('log_player_attack') # 교전 (공격한 경우) 데이터

        attackData = [(match_id, map, log['attacker']['name'], log['attacker']['location']['x'], log['attacker']['location']['y'],
               (pd.to_timedelta(log.timestamp[log.timestamp.find('T')+1:-1]) - startTime).total_seconds(),
              log['attacker']['team_id'])
                for log in attackLog if pd.to_timedelta(log.timestamp[log.timestamp.find('T')+1:-1]) > startTime]

        attackData = pd.DataFrame(attackData, columns=['match_id', 'map', 'name', 
                                                       'loc_x', 'loc_y', 'time', 'team_id'])
        attackData['phase'] = attackData['time'].map(get_phase)
        attack_df = pd.concat([attack_df, attackData], ignore_index = True)
    except:
        print(match_id)

1it [00:01,  1.38s/it]

8f14d554-c335-4801-ba7e-37eee30ffb30


3it [00:07,  2.42s/it]


In [82]:
print(attack_df.shape)
attack_df.head()

(479245, 8)


,match_id,map,name,loc_x,loc_y,time,team_id,phase
0,8cc6c687-657b-48a0-8282-9e6b7e16d2ce,Camp Jackal,angkimozzieeeeee,126706.976562,138285.625,54.43548,1,1
1,8cc6c687-657b-48a0-8282-9e6b7e16d2ce,Camp Jackal,angkimozzieeeeee,126706.976562,138285.625,54.53548,1,1
2,8cc6c687-657b-48a0-8282-9e6b7e16d2ce,Camp Jackal,angkimozzieeeeee,126706.976562,138285.625,54.56848,1,1
3,8cc6c687-657b-48a0-8282-9e6b7e16d2ce,Camp Jackal,angkimozzieeeeee,126706.976562,138285.625,54.63548,1,1
4,8cc6c687-657b-48a0-8282-9e6b7e16d2ce,Camp Jackal,angkimozzieeeeee,126706.976562,138285.625,54.73648,1,1


#### save the battle log data

In [83]:
json_data = attack_df.to_json(orient='records')

In [84]:
json_file_name = f'data/{date}/{date}_btl_data.json'
json_file_name

'data/2024-02-09/2024-02-09_btl_data.json'

In [85]:
with open(json_file_name, 'w') as f:
    json.dump(json_data, f)